___
# Projeto 2 - Ciência dos Dados 2020.2
Grupo: Lucas Kang, Lucas Franco Florentino
___

# Descrição do projeto



O aumento no número de suícios de 200% a 400% nos últimos 20 anos vem se tornando um problema de saúde pública em vários países ao redor do mundo. Dessa forma, várias pesquisas foram desenvolvidas com a intenção de entender esse problema e suas relações externas. Quando relacionado a idade e o sexo com a frequência dos casos de suicídio, existe uma variação interessante de país para país, além disso, no caso dos homens, ocorrem geralmente entre 25 e 35 anos, já nas mulheres entre 18 e 30 anos. Existem alguns indicadores de suicídio, porém, a maioria deles referem ao desemprego, situação financeira do país e consequentemente a presença de doenças psiquiátricas e crônicas. 

   Dessa forma, o projeto tem como função principal observar o relacionamento entre vários indicadores como idade, sexo, situação econômica do país e a geração, com o número de suicídios de um país. Dessa forma, será examinado os resultados e as conexões com o número de suicídios. 

## Objetivo do projeto:

- A partir de diversos fatores como o ano, sexo, faixa etária e situação econômica de um país, é possível prever a quantidade de pessoas que irão cometer suicídio neste ano?  
- O target é quantitativo, já que o objetivo é prever uma informação numérica.

### Importando as bibliotecas:

In [105]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
from math import *
from scipy import *
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import r2_score
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import accuracy_score

### Importando a base de dados:

In [24]:
suicidios = pd.read_csv('suicide_rates.csv')

# apagar depois

## coisas a serem estudadas

- multiple linear regression
- random forest regression
- variável qualitativa precisa ser transformada em dummies
- pd.get_dummies() -- estudar e ler a documentação do comando

# Explicando o dataset:

A base de dados suicide_rates.csv contém um conjunto de informações com cerca de 102 países ao passar de 33 anos. Dessa forma, o código irá analisar uma base de treinamento, e depois, aplicar um teste nos classificadores, onde um deles tem como princípio a regressão múltipla e o outro de #aaaaaa, para então provar sua eficiência em prever a quantidade de suicídios em um ano. 
A base de dados contém os seguintes dados:

- Country: País
- Year: Ano
- Sex: Sexo
- Age: Idade
- Suicides_no: Número de suicídios no país, no ano, no sexo e na faixa etária presente na mesma linha
- Population: População do país, do ano, do sexo e da faixa etária presente na mesma linha
- Suicides/100kpop: Taxa de suicideos a cada 100.000 pessoas com os filtros presentes na mesma linha
- Country-year: País e ano
- HDI for year: Índice de desenvolvimento humano por ano
- Gdp_for_year (): Produto interno bruto por ano em dolar
- Gdp_per_capita (): Produto interno bruto per capita em dolar
- Generation: Geração do indivíduo


Nossa base de dados:
- 

In [25]:
suicidios.head(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


---
# Organizando os dados

Os dados foram organizados em novas colunas e alguns valores foram traduzidos para números que possa ser utilizados pelo modelo para realizar a predição.
Algumas destas novas colunas foram:
- criação de dummies para armazenar a faixa etária de cada linha;
- criação da coluna de classificação econômica;
- traduzir a coluna de geração de valores categóricos para valores numéricos;
- traduzir a coluna de sexo de valores categóricos para valores numéricos.

In [26]:
faixa_etaria = pd.get_dummies(suicidios['age'])
suicidios = pd.concat([suicidios, faixa_etaria], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,0,0,1,0,0,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,0,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,0,1,0,0,0,0


In [27]:
# comparar gdp_per_capita pra determinar se o país naquele ano era considerado Rico, Em desenvolvimento, ou Pobre (e.g: resultado maior que 30000 dol = rico, menor que 5000 dol/pessoa = probre)
# Utilizando a base de dados do IMF, foi feito uma divisão do gdp do país pelo gdp de Luxemburgo do ano de 2020 (USD 109602). 
# Os países que se encontraram com porcentagem abaixo de 30% foram consideradas pobres, de 31% a 60%, em desenvolvimento, e acima de 61%, rico.
# A divisão categórica foi feita a fim de analizar o efeito da situação econômica na mortalidade de pessoas por suicídio.
'''
Legenda:
    Pobre: 3
    Em Desenvolvimento: 2
    Rico: 1
'''
suicidios['classificação econômica'] = np.where((suicidios['gdp_per_capita ($)']/109602)>0.61, 'rico', 
    (np.where((suicidios['gdp_per_capita ($)']/109602)<0.3, 'pobre','em desenvolvimento')))

#dummies da classificação econômica
suicidios = pd.concat([suicidios, pd.get_dummies(suicidios['classificação econômica'])], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years,classificação econômica,em desenvolvimento,pobre,rico
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,1,0,0,0,0,0,pobre,0,1,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,0,1,0,0,0,pobre,0,1,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,1,0,0,0,0,0,pobre,0,1,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,0,0,0,0,0,1,pobre,0,1,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,1,0,0,0,0,pobre,0,1,0


In [28]:
# criação de uma dummies para armazenar a geração em formato numérico
gen = pd.get_dummies(suicidios['generation'])
suicidios = pd.concat([suicidios, gen], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,classificação econômica,em desenvolvimento,pobre,rico,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,pobre,0,1,0,0,0,1,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,pobre,0,1,0,0,0,0,0,0,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,pobre,0,1,0,0,0,1,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,pobre,0,1,0,0,1,0,0,0,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,pobre,0,1,0,1,0,0,0,0,0


In [29]:
# criação de uma dummies para armazenar o sexo em formato numérico
suicidios.loc[suicidios['sex']=='male','sexo'] = 1
suicidios.loc[suicidios['sex']=='female','sexo'] = 0
suicidios['sexo'] = suicidios['sexo'].astype('int64')
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,em desenvolvimento,pobre,rico,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent,sexo
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,0,1,0,0,0,1,0,0,0,1
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,1,0,0,0,0,0,0,1,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,0,1,0,0,0,1,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,0,1,0,0,1,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,1,0,1,0,0,0,0,0,1


In [30]:
#deletar colunas que não serão utilizadas
suicidios.drop('age', inplace = True, axis = 1)
suicidios.drop('year', inplace = True, axis = 1)
suicidios.drop('country-year', inplace = True, axis = 1)
suicidios.drop('HDI for year', inplace = True, axis = 1)
suicidios.drop('generation', inplace = True, axis = 1)
suicidios.drop('sex', inplace = True, axis = 1)
suicidios.drop('suicides/100k pop', inplace = True, axis = 1)

In [31]:
suicidios.head(10)

,country,suicides_no,population,gdp_for_year ($),gdp_per_capita ($),15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,...,em desenvolvimento,pobre,rico,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent,sexo
0,Albania,21,312900,"2,156,624,900",796,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,Albania,16,308000,"2,156,624,900",796,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,1
2,Albania,14,289700,"2,156,624,900",796,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,Albania,1,21800,"2,156,624,900",796,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
4,Albania,9,274300,"2,156,624,900",796,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [32]:
# Eliminando anomalias
suicidios.describe()

,suicides_no,population,gdp_per_capita ($),15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years,em desenvolvimento,pobre,rico,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent,sexo
count,27820.000000,2.782000e+04,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.00000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000,27820.000000
mean,242.574407,1.844794e+06,16866.464414,0.166858,0.166858,0.166858,0.165708,0.166858,0.166858,0.135945,0.841625,0.02243,0.179367,0.098634,0.230338,0.052840,0.210065,0.228756,0.500000
std,902.047917,3.911779e+06,18887.576472,0.372856,0.372856,0.372856,0.371825,0.372856,0.372856,0.342737,0.365099,0.14808,0.383667,0.298175,0.421057,0.223717,0.407362,0.420040,0.500009
min,0.000000,2.780000e+02,251.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,9.749850e+04,3447.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,4.301500e+05,9372.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
75%,131.000000,1.486143e+06,24874.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,22338.000000,4.380521e+07,126352.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


---
---
# Multiple Regression

Tendo a preparação da base de dados completa, podemos iniciar o estudo da relação das variáveis e assim, prever o número de pessoas a cometer suicídio em um ano.

---
Para iniciar a aplicação, é necessário separar os dados independentes dos dados dependentes.

Para isto, "X" foi declarado como sendo as variáveis independentes, e "y" foi definido como o valor dependente. 

In [150]:
X = suicidios[['15-24 years', '25-34 years', '35-54 years', '5-14 years', '55-74 years', '75+ years', 'em desenvolvimento', 'pobre', 'rico', 'Boomers', 'G.I. Generation', 'Generation X', 'Generation Z', 'Millenials', 'Silent', 'sexo']]
y = suicidios['suicides_no']

#fazer X_train e y_train
# Separar o dataset para ser utilizado no teste e no treinamento
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [94]:
modelo = linear_model.LinearRegression().fit(x_train,y_train)
modelo.score(x_train,y_train)

0.0682376753651579

In [95]:
modelo.score(x_test,y_test)

0.07087761492301925

In [96]:
print(modelo.coef_)

[-7.12581258e+13 -7.12581258e+13 -7.12581258e+13 -7.12581258e+13
 -7.12581258e+13 -7.12581258e+13 -4.33612015e+14 -4.33612015e+14
 -4.33612015e+14  4.76150693e+14  4.76150693e+14  4.76150693e+14
  4.76150693e+14  4.76150693e+14  4.76150693e+14  2.66580841e+02]


In [97]:
'''
insert graph depois
'''

'\ninsert graph depois\n'

---
---
# Random Forest Regression

Continuamos a utilizar o "X" e "y" do modelo passado e também a separação do dataset de teste e de treino.
 
Descrever como foi implementado

In [154]:
#n_estimators = número de árvores que serão criadas para o random forest
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 0)
regressor.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=0)

In [155]:
y_pred = regressor.predict(x_test)
y_pred
r2_score(y_test,y_pred)

0.09909443095531889

In [115]:

accuracy_score(y_test,y_pred)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

---
Como pode ser observado na célula anterior, o resultado obtido não é satisfatório, o valor do score é somente 0,101, enquanto que um bom score estaria perto de 1.

Para melhorar os resultados do modelo, foi necessário conduzir um hyperparameter tuning. O hyperparameter tuning é feito para ajustar as configurações do algoritmo e assim melhorar sua performance. 

Normalmente, os parâmetros a serem utilizados são aprendidos durante o treinamento do modelo, mas em hyperparameter tuning, os parâmetros devem ser definidos antes do teste. Alguns destes parâmetros são:
- número de árvores a serem criadas
- número de recursos que serão considerados para cada árvore quando forem dividir um nó

## Aplicando o Hyperparameter Tuning

O Hyperparameter Tuning foi utilizado para otimizar o modelo e obter resultados mais precisos. 

### Um dos métodos utilizados para realizar a busca de parâmetros é através do Random Search Cross Validation

O Random Search Cross Validation consegue utilizar parâmetros diferentes gerados em uma tabela com uma variedade de valores de hiperparâmetros, realizando o K-Fold CV com cada combinação de valores.

K-Fold CV sendo uma técnica em que os dados de treinamento são divididos em K vezes, que são divididas novamente em dobras. O modelo é, então, ajustado K vezes, cada vez treinando o dado de treino em K-1 das dobras e avaliando sua performance de até Kn. 

In [41]:
# Parâmetros que estão sendo utilizados atualmente
regressor.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

Observando a documentação do Random Forest Regressor no Scikit-learn, foi possível diminuir o número de parâmetros significantes para apenas 2:
- n_estimators
- max_features
- max_depth
- min_samples_split
- min_samples_leaf
- boostrap



In [42]:
#Criando o Random Hyperparameter Grid para utilizar o RandomizedSearchCV depois
# número de árvores
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# número de features a serem consideradas para cada split
max_features = ['auto','sqrt']
# máximo número de níveis em cada árvore
max_depth = [int(x) for x in np.linspace(10, 110, num =11)]
max_depth.append(None)
# mínimo número de amostras que são exigidos para cada nó
min_samples_split = [2, 5, 10]
# mínimo número de amostras que são exigidas para cada leaf node
min_samples_leaf = [1, 2, 4]
# método de selecionamento para treinamento de cada árvore
bootstrap = [True, False]

#criando o Random Grid
random_grid = {'n_estimators' : n_estimators, 'max_features' : max_features, 'max_depth' : max_depth, 'min_samples_split' : min_samples_split, 'min_samples_leaf' : min_samples_leaf, 'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [43]:
# Treinando o modelo com o RandomSearchCV
novo_regressor = RandomForestRegressor()

# Busca por parâmetros randomizada, utilizando 3 dobras
# Busca em 100 diferentes combinações
regressor_random = RandomizedSearchCV(estimator = novo_regressor, param_distributions = random_grid, n_iter = 100, cv = 3, verbose = 2, random_state = 42, n_jobs = -1)

regressor_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
# melhores parâmetros a serem utilizados
regressor_random.best_params_

In [ ]:
# Avaliar se os parâmetros utilizados no Random Search foram as melhores para resultarem em um modelo melhor
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

### Avaliar o modelo padrão

In [ ]:
base = RandomForestRegressor(n_estimators = 10, random_state = 42)
base.fit(x_train,y_train)
base_accuracy = evaluate(base, x_test, y_test)

### Avaliar o melhor modelo de acordo com o Random Search Model

In [ ]:
melhor_random = regressor_random.best_estimator_
melhor_accuracy = evaluate(melhor_random, x_test, y_test)

In [ ]:
melhor_pred = melhor_random.predict(x_test)
melhor_pred

In [ ]:
r2_score(y_test,melhor_pred)

### Grid Search

In [ ]:
# Criar a grade de parâmetros baseado nos resultados do random search
param_grade = {
    'bootstrap': [True],
    'max_depth': [30, 40, 50, 80, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [4, 10, 12],
    'n_estimators': [200, 500, 1000, 2000]
}

'''
{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': False}
'''

random_forest = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
procura_grade = GridSearchCV(estimator = random_forest, param_grid = param_grade, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
procura_grade.fit(x_train, y_train)

In [ ]:
procura_grade.best_params_

In [ ]:
novo_melhor = procura_grade.best_estimator_
novo_accuracy = evaluate(novo_melhor, x_test, y_test)

# Bibliografia

_links utilizados:_

- https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
- https://www.imf.org/en/Publications/WEO/weo-database/2021/April/weo-report?c=512,914,612,614,311,213,911,314,193,122,912,313,419,513,316,913,124,339,638,514,218,963,616,223,516,918,748,618,624,522,622,156,626,628,228,924,233,632,636,634,238,662,960,423,935,128,611,321,243,248,469,253,642,643,939,734,644,819,172,132,646,648,915,134,652,174,328,258,656,654,336,263,268,532,944,176,534,536,429,433,178,436,136,343,158,439,916,664,826,542,967,443,917,544,941,446,666,668,672,946,137,546,674,676,548,556,678,181,867,682,684,273,868,921,948,943,686,688,518,728,836,558,138,196,278,692,694,962,142,449,564,565,283,853,288,293,566,964,182,359,453,968,922,714,862,135,716,456,722,942,718,724,576,936,961,813,726,199,733,184,524,361,362,364,732,366,144,146,463,528,923,738,578,537,742,866,369,744,186,925,869,746,926,466,112,111,298,927,846,299,582,487,474,754,698,&s=NGDPD,PPPGDP,NGDPDPC,PPPPC,&sy=2010&ey=2021&ssm=0&scsm=1&scc=0&ssd=1&ssc=0&sic=1&sort=subject&ds=.&br=1
- https://www.w3schools.com/python/python_ml_multiple_regression.asp
- https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
- https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74